
<h1 style="font-size:30px;">1) Preparing the data</h1>

In [114]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import string #to later delete punctuautions from my text

In [115]:
#loading the data:

df1 = pd.read_csv('reviews_train.csv') #reviews
df2 = pd.read_csv('sw.txt') #stop words

In [116]:
#lowercase:

for i in range(len(df1)):
    df1['Review'][i] = df1['Review'][i].lower()
    df1['Label'][i] = df1['Label'][i].lower()
    
for j in range(len(df2)):
    df2['about'][j] = df2['about'][j].lower()

In [117]:
#convert to numpy array

df1 = df1.values
df2 = df2.values

In [118]:
#seperate the 'pos' and 'neg' classes:

positive = 0
negative = 0

positive_reviews = np.array([])
negative_reviews = np.array([])
for i in range(len(df1)):
    if df1[i, 1] == 'pos':
        positive += 1
        positive_reviews = np.append(positive_reviews, df1[i, 0])
    if df1[i, 1] == 'neg':
        negative += 1
        negative_reviews = np.append(negative_reviews, df1[i, 0])

stop_words = np.array([])
for j in range(len(df2)):
    stop_words = np.append(stop_words, df2[j, 0])

In [119]:
#eliminating the punctuations:

for i in range(len(positive_reviews)):
    positive_reviews[i] = positive_reviews[i].translate(str.maketrans('', '', string.punctuation))
    
    
for i in range(len(negative_reviews)):
    negative_reviews[i] = negative_reviews[i].translate(str.maketrans('', '', string.punctuation))
    
    

In [120]:
import nltk


#all the positive words:
positive_words = np.array([])
for i in range(len(positive_reviews)):
    positive_words = np.append(positive_words, nltk.tokenize.word_tokenize(str(positive_reviews[i])))
#     positive_words = np.append(positive_words, positive_reviews[i].split()) #but using "nltk" methods also seperates punctuations, etc. Hence, it is waaay better than using ".split()"
    
    
#all the negative words:    
negative_words = np.array([])
for i in range(len(negative_reviews)):
    negative_words = np.append(negative_words, nltk.tokenize.word_tokenize(str(negative_reviews[i])))
#     negative_words = np.append(negative_words, negative_reviews[i].split())

In [121]:
# # Eliminationg words with less than a size of 1 (like 'a', '6', etc.):

# positive_words_final = np.array([])
# for j in range(len(positive_words)):
#     if len(positive_words[j]) > 1:
#         positive_words_final = np.append(positive_words_final, positive_words[j])
# positive_words = positive_words_final

# negative_words_final = np.array([])
# for j in range(len(negative_words)):
#     if len(negative_words[j]) > 1:
#         negative_words_final = np.append(negative_words_final, negative_words[j])
# negative_words = negative_words_final

In [122]:
#eliminating the stop_words:

positive_words = np.array([t for t in positive_words if t not in stop_words])
negative_words = np.array([t for t in negative_words if t not in stop_words])

In [123]:
#counting each word's frequency for both 'pos' and 'neg' classes:

import collections

positive_dic = collections.Counter(positive_words)
negative_dic = collections.Counter(negative_words)    

In [124]:
#number of elements in both 'pos' and 'neg' classes:

num_positive = len(positive_words)
num_negative = len(negative_words)
total = num_positive + num_negative

p_class_positive = positive / (positive + negative)
p_class_negative = negative / (positive + negative)

In [125]:
# #thus, in order to get p(word|class):
# positive_dic = positive_dic / num_positive
# negative_dic = negative_dic / num_negative

# positive_dic_keys = positive_dic.keys().values()
# positive_dic_keysfor 

In [126]:
num_dic_positive = len(positive_dic)
num_dic_negative = len(negative_dic)


<h1 style="font-size:30px;">2,3) Classification and laplace smoothing:</h1>

In [127]:
df3 = pd.read_csv('reviews_test.csv') #test data

In [128]:
#lowercase:

for i in range(len(df3)):
    df3['Review'][i] = df3['Review'][i].lower()
    df3['Label'][i] = df3['Label'][i].lower()

In [129]:
#convert to numpy array

df3 = df3.values

In [130]:
a = 1

total_probability_positive = 1
total_probability_negative = 1

pr_positive = np.array([])
pr_negative = np.array([])


for i in range(len(df3)):
    
    review = df3[i, 0]
    review = review.translate(str.maketrans('', '', string.punctuation))
    words = nltk.tokenize.word_tokenize(str(review))
    words = np.array([t for t in words if t not in stop_words])
    
#     words_final = np.array([])
#     for j in range(len(words)):
#         if len(positive_words[j]) > 1:
#             words_final = np.append(words_final, words[j])
#     words = words_final
    
    d = int(np.unique(words).shape[0])
    for word in words:
        probability_positive = (positive_dic[str(word)] + a) / (num_dic_positive + a*d) #(num_dic_positive + a*(num_dic_positive + num_dic_negative)) 
        total_probability_positive *= probability_positive
        probability_negative = (negative_dic[str(word)] + a) / (num_dic_negative + a*d)
        total_probability_negative *= probability_negative

    pr_positive = np.append(pr_positive, (total_probability_positive * p_class_positive))
    pr_negative = np.append(pr_negative, (total_probability_negative * p_class_negative))
    
    probability_positive = 1
    total_probability_positive = 1
    probability_negative = 1
    total_probability_negative = 1

    
pred_labels = np.array([])
for i in range(len(df3)):
    if (pr_positive[i] > pr_negative[i]):
        pred_labels = np.append(pred_labels, np.array([1])) #positive
    if (pr_positive[i] < pr_negative[i]):
        pred_labels = np.append(pred_labels, np.array([0])) #negative
        
        
        
true_labels = np.array([])
for i in range(len(df3)):
    if df3[i][1] == 'pos':
        true_labels = np.append(true_labels, np.array([1]))
    if df3[i][1] == 'neg':
        true_labels = np.append(true_labels, np.array([0]))

In [131]:
pr_positive

array([3.45845952e-323, 1.54267923e-268, 1.91124262e-231, 9.42769513e-079,
       7.64474973e-100, 1.98664521e-049, 2.61964091e-073, 2.09662824e-128,
       6.13207990e-102, 3.57038933e-067, 5.58970779e-068, 1.99494010e-159,
       4.94595932e-053, 3.16088080e-123, 6.31930742e-150, 7.48980845e-159,
       3.77658413e-162, 7.25690143e-047, 1.02855903e-057, 3.13835365e-107])

In [132]:
pr_negative

array([1.89796855e-314, 2.03998437e-259, 1.24996139e-229, 1.63116311e-078,
       1.31279489e-099, 1.43354832e-050, 1.88174380e-073, 1.51857150e-127,
       1.85120776e-101, 1.27157810e-065, 3.86084623e-065, 1.36756365e-154,
       5.20680552e-054, 1.60063992e-120, 1.47248757e-143, 3.72015871e-153,
       2.78522630e-157, 1.11837448e-046, 2.45831565e-055, 1.57028027e-104])

In [133]:
true_labels

array([0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 1.])

In [134]:
pred_labels

array([0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0.])

In [135]:
np.concatenate((pred_labels.reshape(len(pred_labels), 1), true_labels.reshape(len(true_labels), 1)), axis = 1)

array([[0., 0.],
       [0., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 1.],
       [1., 1.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [1., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [136]:
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(true_labels, pred_labels)
print(cm)

[[10  1]
 [ 7  2]]


In [137]:
ac = accuracy_score(true_labels, pred_labels)
ac #the correct predictions

0.6